# Case Study 2 - Spatial projection via modelled data
## Description 
Estimate soil pH and electrical conductivity at 45 cm depth across the Llara-Campey farm sites based on values collected from soil samples across the farm. 

This is an example of predicting a spatial variable across a region based on a calculation dependent on data for multiple other spatial variables. Comparable cases could include:
- Analyse expected yield for different chickpea and wheat varieties if these were grown across Western Australia under actual seasonal weather conditions, so I can estimate the benefit associated with optimised varietal selection. 
- Predict population levels for a pest species across a region based on a function derived from Case Study 1 and environmental context for the region at the time and over the previous month, including weather (temperature, rainfall, humidity - all xyt), lunar phase (t) and greenness (xyt - see https://portal.tern.org.au/metadata/TERN/8542d90e-6e20-4ad8-b30d-0a171b61d3f5).

## Data sources
- **BradGinns_SOIL2004_SoilData.csv** - Soil measurements from the University of Sydney Llara Campey farm site from 2004, corresponding to sites L1, L3 and L4 describing mid-depth, soil apparent electrical conductivity (ECa), GammaK, Clay, Silt, Sand, pH and soil electrical conductivity (EC)
- **Llara_Campey_field_boundaries_poly.shp** - Field boundary shapes for the University of Sydney Llara Campey farm site

## Dependencies
- This notebook requires Python 3.10 or higher
- Install relevant Python libraries with: **pip install mccn-engine rocrate rioxarray pykrige**
- Installing mccn-engine will install other dependencies

## Overview
1. Select soil sample measurements for pH or EC at 45cm depth
2. Split sample measurements into 80% subset to model interpolated layers and 20% to test interpolated layers
3. Generate STAC metadata for layers
4. Load data cube
5. Interpolate pH and EC across site using the 80% subset and three different 2D interpolation methods from rioxarray (nearest, linear and cubic) and one from pykrige (linear)
6. Calculate the error between each layer of interpolated values and measured values for the 20% setaside for testing
7. Compare the mean and standard deviation of the errors for each interpolation method
8. Clean up and package resuls as RO-Crate

## Notes
- The granularity of variability in soil data significantly compromises all methods
- Depending on the 80/20 split, different methods may appear more reliable, but the pykrige linear method is most often best

## Imports

In [ ]:
import rioxarray
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import shutil
import pystac
import pystac_client
import subprocess
import json
import pykrige

from stac_generator.factory import StacGeneratorFactory
from stac_generator.core.base.generator import StacSerialiser
from stac_generator.core.base.schema import StacCollectionConfig, ColumnInfo
from stac_generator.core.raster.schema import RasterConfig, BandInfo
from stac_generator.core.vector.schema import VectorConfig
from stac_generator.core.point.schema import PointConfig

from mccn.client import MCCN

from rocrate.rocrate import ROCrate
from rocrate.model import ComputationalWorkflow, SoftwareApplication, File, Dataset, ContextEntity, Person

## Data Paths
Locations for folders and files used in this notebook.

In [ ]:
# Paths to current folder and scratch folder for working files
current_folder = os.getcwd()
source_folder = os.path.join(current_folder, "source_data")
stac_folder = os.path.join(current_folder, "stac")
scratch_folder = os.path.join(current_folder, "scratch")
results_folder = os.path.join(current_folder, "results")

# Create clean folders for outputs
for f in [stac_folder, scratch_folder, results_folder]:
    if os.path.exists(f):
        shutil.rmtree(f)
    os.makedirs(f)

# Path to this notebook
notebook = "MCCN-CASE 2.ipynb"

# Paths to source files
soil_source = os.path.join(source_folder, "BradGinns_SOIL2004_SoilData.csv")
boundary_source = os.path.join(source_folder, "Llara_Campey_field_boundaries_poly.shp")

# Paths to working files
soil_datafile = os.path.join(scratch_folder, "SoilData.csv")

# Paths to results files
datacube_initial = os.path.join(results_folder, "Projection_study_initial.dc")
datacube_final = os.path.join(results_folder, "Projection_study_final.dc")
soil_training_file = os.path.join(results_folder, "Projection_soil_training.csv")
soil_sample_file = os.path.join(results_folder, "Projection_soil_sample.csv")
soil_error_file = os.path.join(results_folder, "Projection_soil_error_results.csv")

# Path to generated RO-Crate results package
ro_crate = "MCCN-CASE 2.RO-Crate.zip"
if os.path.exists(ro_crate):
    os.remove(ro_crate)

## Prepare soil data
Read data from 2004 soil dataset. Limit to samples at 45cm depth and for pH or EC. Save data for reference. Split data into a training set for estimating the variables and a smaller set (20% of records) for assessing errors.

In [ ]:

# Read soil_source and keep elements to be used in analysis
soil_data = pd.read_csv(soil_source, encoding="UTF8")[["Site", "Lat", "Long", "mid-depth", "pH", "EC"]].rename(columns={"mid-depth": "Depth"})
soil_data = soil_data[soil_data["Depth"] == 0.45].drop(columns=["Depth"])
soil_data.to_csv(soil_datafile, index=False)

# Select random subset to test results of interpolation methods
soil_data_sample = soil_data.sample(frac=0.2).rename(columns={"pH": "pHSample", "EC": "ECSample"})
soil_data_sample.to_csv(soil_sample_file, index=False)

# Treat all other samples as training data
soil_data_training = soil_data[~soil_data.index.isin(soil_data_sample.index)].rename(columns={"pH": "pHTraining", "EC": "ECTraining"})
soil_data_training.to_csv(soil_training_file, index=False)

## Generate configuration files for STAC collection

In [ ]:
# Configuration for collection as a whole.
collection_config = StacCollectionConfig(
    id="SoilStudy",
    title="Datasets for soil case study",
    description="STAC records for accessing datasets to explore as part of the MCCN case study 2 relating to interpolation of soils data",
    license="CC-BY-4.0",
)

# List of configurations for individual data layers.
configurations = [
    
    VectorConfig(
        id="Site_Boundary",
        location=boundary_source,
        collection_date="2024-12-31",
        collection_time="00:00:00"
    ),
    
    PointConfig(
        id="Soil data",
        location=soil_datafile,
        collection_date="2004-12-31",
        collection_time="00:00:00",
        X="Long",
        Y="Lat",
        #Z="Depth",
        column_info=[
            ColumnInfo(name="pH", description=f"pH measurement for given location and depth"),
            ColumnInfo(name="EC", description=f"EC measurement for given location and depth"),
        ]
    ),

    PointConfig(
        id="Soil data sample",
        location=soil_sample_file,
        collection_date="2004-12-31",
        collection_time="00:00:00",
        X="Long",
        Y="Lat",
        #Z="Depth",
        column_info=[
            ColumnInfo(name="pHSample", description=f"pH measurement for given location and depth"),
            ColumnInfo(name="ECSample", description=f"EC measurement for given location and depth"),
        ]
    ),

    PointConfig(
        id="Soil data training",
        location=soil_training_file,
        collection_date="2004-12-31",
        collection_time="00:00:00",
        X="Long",
        Y="Lat",
        #Z="Depth",
        column_info=[
            ColumnInfo(name="pHTraining", description=f"pH measurement for given location and depth"),
            ColumnInfo(name="ECTraining", description=f"EC measurement for given location and depth"),
        ]
    )

]

# Build the generator using the configurations.
generator = StacGeneratorFactory.get_collection_generator(
    source_configs=configurations,
    collection_config=collection_config
)

# Serialise the STAC collection. This will generate the collection JSON file and item JSON files for each layer.
serialiser = StacSerialiser(generator, stac_folder)
serialiser()

## DataCube Load
Load data for all STAC items for the site boundaries and soil data (including the training and sample subsets).

In [ ]:
# Using the locally generated collection
endpoint = os.path.join(stac_folder, "collection.json")
client = MCCN(endpoint, shape=(200,100), nodata={"Site_Boundary": 0}, nodata_fallback=np.nan)
ds = client.load()
MCCN.to_netcdf(ds, datacube_initial)

# The xarray includes a time dimension with a single position. Removing this dimension simplifies the indexing in all later steps.
boundary_data = np.fmax(ds["Site_Boundary"].isel(time=0), ds["Site_Boundary"].isel(time=1))
ds = ds.isel(time=0)
ds["Site_Boundary"] = boundary_data

# Display DataSet.
ds

## Interpolate data
For each variable, use three different 2D interpolation methods from rioxarray (nearest, linear and cubic) and one from pykrige (linear) to estimate values at other locations in area. Crop these estimates to the boundaries of the farm. 

Note that the rioxarray linear and cubic interpolations are restricted to the convex hull of the available points, so some of the farm areas are not included in their estimates.

Repeat each calculation for the training datasets.

In [ ]:
soil_data["Long"]

In [ ]:
variables = {"pH": [], "pHTraining": [], "EC": [], "ECTraining": []}

for v in variables.keys():
    ds[v].rio.write_nodata(np.nan, encoded=True, inplace=True)
    variables[v].append(v)
    for m in ["nearest", "linear", "cubic", "krige_linear"]:
        name = f"{v}_{m}"
        variables[v].append(name)
        if m.startswith("krige"):
            krige_data = soil_data_training if "Training" in v else soil_data
            OK = pykrige.ok.OrdinaryKriging(krige_data["Long"], krige_data["Lat"], krige_data[v], verbose=False, enable_plotting=False)
            ds[name] = xr.DataArray(OK.execute("grid", list(ds.x), list(ds.y))[0], coords={"y": ds.y, "x": ds.x})
        else:
            ds[name] = ds[v].rio.interpolate_na(method=m)
        ds[name] = ds[name].where(ds["Site_Boundary"] > 0, np.nan)

    ds[variables[v]].to_array().plot(x="x", col="variable", col_wrap=5)

## Calculate errors
Use the 20% sample data set aside earlier to test the interpolations. Compare values at each sample point and calculate error.

In [ ]:
s = soil_data_sample.copy()
for _, row in s.iterrows():
    for c in ["pH", "EC"]:
        cs = f"{c}Sample"
        ct = f"{c}Training"
        for v in variables[ct]:
            if not v.endswith("Training"):
                ve = f"{v}_error"
                if ve not in soil_data_sample.columns:
                    soil_data_sample[ve] = np.nan
                estimate = ds[v].sel(x=row["Long"], y=row["Lat"], method="nearest").values
                soil_data_sample.loc[(soil_data_sample["Long"] == row["Long"]) & (soil_data_sample["Lat"] == row["Lat"]), [v, ve]] = [estimate, abs(estimate - row[cs])]

soil_data_sample

## Compare methods
Calculate the mean error and standard deviation of the errors for each method for each variable.

This comparison is of limited value. In reality, the variables in question seem to vary significantly at fine scales, so all interpolation methods have poor results.

Depending on which points were selected for the sample, different methods may seem to fit the sample better. In most cases, the pykrige linear model has the best fit. 

In [ ]:
errors = pd.DataFrame(columns=["variable", "method", "mean", "std"])
for c in soil_data_sample.columns:
    if "error" in c:
        variable = c[0:2]
        method = " ".join(c.split("_")[1:-1])
        mean = soil_data_sample[c].mean()
        std = soil_data_sample[c].std()
        errors.loc[len(errors)] = [variable, method, mean, std]
        #print(f"{c} - mean error: {soil_data_sample[c].mean():.3f}, error standard error: {soil_data_sample[c].std():.3f}")

errors

## Cleanup
Clean up temporary files. Write Xarray and Pandas outputs to the results folder. Package everything as an RO-Crate.

In [ ]:
# Clean up scratch folder
if os.path.exists(scratch_folder):
    shutil.rmtree(scratch_folder)

# Save output datasets
MCCN.to_netcdf(ds, datacube_final)
errors.to_csv(soil_error_file)

# Package notebook, source data and outputs as an RO-Crate
crate = ROCrate()
ardc = crate.add(ContextEntity(crate, "https://ror.org/038sjwq14", properties={
    "@type": "Organization",
    "name": "Australian Research Data Commons"
}))
appn = crate.add(ContextEntity(crate, "https://ror.org/02zj7b759", properties={
    "@type": "Organization",
    "name": "Australian Plant Phenomics Network"
}))
dgh = crate.add(Person(crate, "https://orcid.org/0000-0001-6492-4016", properties={
    "name": "Donald Hobern",
    "email": "donald.hobern@adelaide.edu.au",
    "affiliation": {"@id": appn.id},
    "jobTitle": "APPN Data Management Director",
}))
food_security = crate.add(ContextEntity(crate, "https://doi.org/10.47486/DC105", properties={
    "@type": "Grant",
    "name": "ARDC Project Code DC105: Multi-Scalar Crop Characterisation Network (MCCN)",
    "funder": {"@id": ardc.id}
}))
mccn = crate.add(ContextEntity(crate, "https://doi.org/10.26292/8679d473", properties={
    "@type": "ResearchProject",
    "name": "Multi-Scalar Crop Characterisation Network (MCCN)",
    "funding": {"@id": food_security.id},
    "contactPoint": {"@id": dgh.id},
}))
cc_by = crate.add(ContextEntity(crate, "https://creativecommons.org/licenses/by/4.0/", properties={
    "@type": "License",
    "name": "Creative Commons Attribution 4.0 International (CC BY 4.0)",
}))
crate_properties = crate.default_entities[0].properties()
crate_properties |= {
    "name": "MCCN Case Study 2 - source data, notebook and results",
    "description": "This RO-Crate is a packaged version of the Jupyter notebook for Case Study 2 of the Multiscalar Crop Characterisation Network (MCCN) project. It demonstrates the functionality of the Python packages pystac-generator and mccn-engine developed as part of this project.",
    "license": {"@id": cc_by.id},
    "maintainer": {"@id": mccn.id},
}
crate.add(File(crate, source=(soil_source := os.path.relpath(soil_source, current_folder)), dest_path=soil_source, properties={
    "name": "Soil measurements from 2004, corresponding to sites L1, L3 and L4 describing mid-depth, soil apparent electrical conductivity (ECa), GammaK, Clay, Silt, Sand, pH and soil electrical conductivity (EC)",
    "encodingFormat": "text/csv",
    "producer": "usyd.edu.au",
    "citation": "University of Sydney",
    "license": {"@id": cc_by.id},
    "url": {"@id": "https://object-store.rc.nectar.org.au/v1/AUTH_2b454f47f2654ab58698afd4b4d5eba7/mccn-test-data/LlaraCampey/soil_data.csv"}
}))
crate.add(File(crate, source=(boundary_source := os.path.relpath(boundary_source, current_folder)), dest_path=boundary_source, properties={
    "name": "Shape file of outer boundary for wheat experiments conducted at Llara farm site L1, L3, L4 in 2017, 2019 and 2021 measuring yield, elevation and moisture",
    "encodingFormat": "application/octet-stream",
    "producer": "usyd.edu.au",
    "citation": "University of Sydney",
    "license": {"@id": cc_by.id},
    "url": {"@id": "https://object-store.rc.nectar.org.au/v1/AUTH_2b454f47f2654ab58698afd4b4d5eba7/mccn-test-data/LlaraCampey/boundary_wgs84.zip"}
}))
crate.add(File(crate, source=(datacube_initial := os.path.relpath(datacube_initial, current_folder)), dest_path=datacube_initial, properties={
    "name": "Xarray datacube as loaded from source data files by MCCN Case Study 2",
    "license": {"@id": cc_by.id},
    "encodingFormat": "application/x-netcdf",
    "producer": {"@id": appn.id},
}))
crate.add(File(crate, source=(datacube_final := os.path.relpath(datacube_final, current_folder)), dest_path=datacube_final, properties={
    "name": "Xarray datacube following processing by MCCN Case Study 2",
    "license": {"@id": cc_by.id},
    "encodingFormat": "application/x-netcdf",
    "producer": {"@id": appn.id},
}))
crate.add(File(crate, source=(soil_training_file := os.path.relpath(soil_training_file, current_folder)), dest_path=soil_training_file, properties={
    "name": "CSV table of soil data selected to build kriged layers",
    "license": {"@id": cc_by.id},
    "encodingFormat": "text/csv",
    "producer": {"@id": appn.id},
}))
crate.add(File(crate, source=(soil_sample_file := os.path.relpath(soil_sample_file, current_folder)), dest_path=soil_sample_file, properties={
    "name": "CSV table of soil data selected to test kriged layers",
    "license": {"@id": cc_by.id},
    "encodingFormat": "text/csv",
    "producer": {"@id": appn.id},
}))
crate.add(File(crate, source=(soil_error_file := os.path.relpath(soil_error_file, current_folder)), dest_path=soil_error_file, properties={
    "name": "CSV table of differences/errors when comparing test soil data with kriged layers",
    "license": {"@id": cc_by.id},
    "encodingFormat": "text/csv",
    "producer": {"@id": appn.id},
}))
mccn_engine = crate.add(SoftwareApplication(crate, "https://github.com/aus-plant-phenomics-network/mccn-engine", properties={
    "name": "MCCN-Engine",
    "description": "Python library for loading and combining STAC-described assets, into an xarray datacube",
    "maintainer": {"@id": mccn.id},
}))
stac_generator = crate.add(SoftwareApplication(crate, "https://github.com/aus-plant-phenomics-network/stac-generator", properties={
    "name": "STAC Generator",
    "description": "Python library that combines automatically extracted geospatial information from raw assets and other user-provided metadata to build a STAC-compliant metadata record for further use",
    "maintainer": {"@id": mccn.id},
}))
nb = crate.add(ComputationalWorkflow(crate, source=notebook, dest_path=notebook, properties={
    "name": "MCCN Case Study 2 Notebook: Spatial projection via modelled data",
    "encodingFormat": "application/x-ipynb+json",
    "creator": {"@id": dgh.id},
    "maintainer": {"@id": mccn.id},
    "license": {"@id": cc_by.id},
    "softwareRequirements": [
        {"@id": mccn_engine.id},
        {"@id": stac_generator.id},
    ],
    "input": [
        { "@id": soil_source},
        { "@id": boundary_source},
    ],
    "output": [
        { "@id": datacube_initial},
        { "@id": datacube_final},
        { "@id": soil_training_file},
        { "@id": soil_sample_file},
        { "@id": soil_error_file},
    ],
}))
crate_properties |= {
    "name": "MCCN Case Study 2 - source data, notebook and results",
    "description": "This RO-Crate is a packaged version of the Jupyter notebook for Case Study 2 of the Multiscalar Crop Characterisation Network (MCCN) project. It demonstrates the functionality of the Python packages pystac-generator and mccn-engine developed as part of this project.",
    "license": {"@id": cc_by.id},
    "maintainer": {"@id": mccn.id},
    "mainEntity": {"@id": nb.id},
}
crate.write_zip(ro_crate)